In [2]:
from utilities.extract import extract_text_and_images

extract_text_and_images('data/sixth_social_chapter1.pdf', 'extracted')

MuPDF error: library error: FT_New_Memory_Face(LNWSSX+NotoSerif-Italic): unknown file format

✅ Extraction complete. Text in 'extracted\text', Images in 'extracted\images'


In [3]:
from sentence_transformers import SentenceTransformer


In [4]:
from utilities.chunk import process_text_files_with_metadata

In [5]:

# Load the SentenceTransformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

In [6]:
process_text_files_with_metadata("extracted/text/",model)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

✅ Processed 26 files. Data saved to 'text_chunks_with_embeddings.json'


In [7]:
from utilities.images import process_images_with_metadata

In [8]:
# Load a CLIP-based model for image embeddings
model_img = SentenceTransformer("clip-ViT-B-32")

# Process images from the folder (from Step 1 extraction)
process_images_with_metadata("extracted/images", model_img)

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


✅ Processed 22 images and saved metadata to 'image_embeddings_with_metadata.json'


In [9]:
from utilities.link import link_text_and_images
import json

In [10]:
# Example usage:
linked = link_text_and_images("text_chunks_with_embeddings.json", "image_embeddings_with_metadata.json")
print(json.dumps(linked, indent=2))

{
  "1": {
    "text_chunks": [
      {
        "source_file": "page_1.txt",
        "page_number": 1,
        "chunk_id": 0,
        "text_chunk": "1\nIntroduction \u2013 Why Social Science?\nIntroduction\nWhy Social Science?\nFamily \nand \nCommunity\nGovernance\nLocal Government \nin Rural Areas\nSocial \nScience\nThe Value \nof Work\nLocal Government \nin Urban Areas\nLocating \nPlaces on the \nEarth\nOceans and \nContinents\nTHEME E\nEconomic  \nLife Around  \nUs\nThe Beginnings of \nIndian Civilisation\nTHEME B\nTapestry \nof the Past\nEconomic \nActivities \nAround Us\nUnity in \nDiversity, \nor \u2018Many in \nthe One\u2019\nTHEME C\nOur Cultural \nHeritage and \nKnowledge \nTraditions\nLa",
        "embedding": [
          0.02339058369398117,
          0.0062782675959169865,
          -0.06834102421998978,
          0.06300637125968933,
          0.03319048509001732,
          0.025606464594602585,
          -0.02419416792690754,
          -0.0737328752875328,
          -0.05

In [11]:
from utilities.vectordb import build_faiss_index, build_faiss_index_for_images, search_text_index

In [12]:
# Build the FAISS index for text embeddings.
text_index, text_metadata = build_faiss_index("text_chunks_with_embeddings.json")

✅ Indexed 83 text items with dimension 384


In [13]:
# Build the FAISS index for image embeddings.
image_index, image_metadata = build_faiss_index_for_images("image_embeddings_with_metadata.json")

✅ Indexed 22 image items with dimension 512


In [14]:
# Example usage (assuming model_text from Step 2 is already loaded):
query = "Explain the map and how to use it."
results = search_text_index(query, text_index, text_metadata, model)
print("Top text results:")
for res in results:
    print(f"Page: {res.get('page_number')} | Chunk ID: {res.get('chunk_id')}")
    print(res.get("text_chunk")[:150] + "...\n")


Top text results:
Page: 8 | Chunk ID: 2
ws in the top 
right corner of the map; we will soon see how they point to 
some specific directions and make maps even more helpful.
Fig. 1.1. A map ...

Page: 23 | Chunk ID: 1
ll or large. The main components of maps are 
distance, direction and symbols.
	
Æ
Every place on the Earth has a location which can be 
precisely def...

Page: 9 | Chunk ID: 2
book or collection of maps.
As you will discover, there are several kinds of maps —
	
physical maps, which mainly show some natural 
features such as...

Page: 9 | Chunk ID: 3
formation 
(examples include Fig. 6.3 and Fig. 8.1 in this textbook).
In addition, there are three important components of 
maps—distance, direction a...

Page: 9 | Chunk ID: 1
d a few 
neighbouring buildings. At the end, compare all the maps 
and discuss.
A Map and Its Components
From this simple example, we understand that ...



In [15]:
def link_text_with_images(text_result, image_metadata):
    """
    For a given text result, find all images on the same page.
    
    Args:
      text_result (dict): A single text chunk metadata item.
      image_metadata (list): List of image metadata items.
      
    Returns:
      List of image metadata items that have the same page_number.
    """
    page = text_result.get("page_number")
    linked_images = [img for img in image_metadata if img.get("page_number") == page]
    return linked_images




In [16]:
# For the first text result, get linked images:
if results:
    first_result = results[0]
    linked_images = link_text_with_images(first_result, image_metadata)
    print(f"Linked images on page {first_result.get('page_number')}:")
    for img in linked_images:
        print(img.get("image_file"))

Linked images on page 8:
page_8_img_1.png
